In [1]:
from tools import *

from tqdm import tqdm
import json
import requests
from tqdm import tqdm

import json
import requests

import tools 
from typing import List, Dict, Tuple


In [2]:
stats = tools.clear_all_graph_data()


🗑️  Starting to clear all graph data...
   Deleting all relationships...
   Deleting all nodes...
   Cleaning up GDS graphs...
   - No 'entity' graph to drop
   Dropping all indexes...
   Dropping all constraints...
✅ Graph cleanup completed!
   - Deleted 0 relationships
   - Deleted 0 nodes
   - Remaining nodes: 0


In [3]:
url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)

In [4]:
def chunk_into_books(text: str) -> List[str]:
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [5]:
token_count = [num_tokens_from_string(el) for el in books]
print(
    f"""There are {len(token_count)} books with token sizes:
- avg {sum(token_count) / len(token_count)}
- min {min(token_count)}
- max {max(token_count)}
"""
)

There are 24 books with token sizes:
- avg 6515.208333333333
- min 4459
- max 10760



In [6]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [7]:
len(chunked_books[0])

22

In [8]:
from typing import List, Dict, Tuple

ENTITY_TYPES = "PERSON,ORGANIZATION,LOCATION,GOD,EVENT,CREATURE,WEAPON_OR_TOOL"

def extract_entities(text: str) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract entities and relationships from text using Bedrock
    
    Args:
        text: Input text to analyze
        
    Returns:
        Tuple of (entities, relationships) as lists of dictionaries
    """
    # Construct prompt
    messages = [
        {"role": "user", "content": tools.create_extraction_prompt(ENTITY_TYPES, text)}
    ]
    
    # Make the LLM call using Bedrock (model parameter is ignored)
    output = tools.chat(messages, model="gpt-4o")  # Will use Bedrock instead
    
    # Parse output and return both entities and relationships
    return tools.parse_extraction_output(output)

In [9]:
number_of_books = 1
for book_i, book in enumerate(
    tqdm(chunked_books[:number_of_books], desc="Processing Books")
):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        neo4j_driver.execute_query(
            tools.import_nodes_query,
            data=nodes,
            book_id=book_i,
            text=chunk,
            chunk_id=chunk_i,
        )
        """
        MERGE (b:Book {id: $book_id})
        MERGE (b)-[:HAS_CHUNK]->(c:__Chunk__ {id: $chunk_id})
        SET c.text = $text
        WITH c
        UNWIND $data AS row
        MERGE (n:__Entity__ {name: row.entity_name})
        SET n:$(row.entity_type),
            n.description = coalesce(n.description, []) + [row.entity_description]
        MERGE (n)<-[:MENTIONS]-(c)
        """

        neo4j_driver.execute_query(
            tools.import_relationships_query, data=relationships
        )
        """
        UNWIND $data AS row
        MERGE (s:__Entity__ {name: row.source_entity})
        MERGE (t:__Entity__ {name: row.target_entity})
        CREATE (s)-[r:RELATIONSHIP {description: row.relationship_description, strength: row.relationship_strength}]->(t)
        """

Book 0:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MUSE;GOD;The Muse is a divine entity invoked by the narrator to tell the story of the hero's journey)
|
("entity";TROY;LOCATION;A famous town that was sacked by the hero mentioned in the text)
|
("entity";HYPERION;GOD;The Sun-god whose cattle were eaten by the hero's men, leading to their demise)
|
("entity";JOVE;GOD;Father of the daughter being addressed by the narrator)
|
("entity";ULYSSES;PERSON;The ingenious hero who traveled far and wide after sacking Troy, and was detained by Calypso while trying to return home)
|
("entity";CALYPSO;GOD;A goddess who detained Ulysses in a large cave and wanted to marry him)
|
("entity";TELEMACHUS;PERSON;Someone who issued a challenge to the suitors, mentioned in the title)
|
("entity";MINERVA;GOD;A deity who visited Ithaca, mentioned in the title)
|
("entity";ITHACA;LOCATION;A place visited by Minerva, likely Ulysses' homeland)
|
("relationship";ULYSSES;TROY;Ulysses


Book 0:   5%|▍         | 1/22 [00:07<02:36,  7.45s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";ITHACA;LOCATION;The homeland where Ulysses was trying to return to after being detained by Calypso)
|
("entity";NEPTUNE;GOD;A god who persecuted Ulysses and prevented him from returning home, later visited the Ethiopians to accept a sacrifice)
|
("entity";ETHIOPIANS;PERSON;People who live at the world's end, divided into two halves looking East and West, and who offered Neptune a hecatomb)
|
("entity";OLYMPIAN JOVE;GOD;The sire of gods and men who hosted a meeting of gods and spoke about Aegisthus)
|
("entity";AEGISTHUS;PERSON;A man who had an affair with Agamemnon's wife and killed Agamemnon, later killed by Orestes)
|
("entity";ORESTES;PERSON;Agamemnon's son who killed Aegisthus)
|
("entity";AGAMEMNON;PERSON;A man who was killed by Aegisthus and whose wife was seduced by Aegisthus, father of Orestes)
|
("entity";OLYMPUS;LOCATION;The house of Olympian Jove where the gods met)
|
("relationship";NEPTUNE;U


Book 0:   9%|▉         | 2/22 [00:15<02:36,  7.82s/it]

...


Book 0:  91%|█████████ | 20/22 [03:19<00:21, 10.98s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;The son of Ulysses who states his father is dead and gone, recognizes Mentes was actually the goddess, and retires to his high room in a tower)
|
("entity";MENTES;PERSON;The disguise taken by the goddess Minerva, described as the son of Anchialus and chief of the Taphians)
|
("entity";ANCHIALUS;PERSON;The supposed father of Mentes (Minerva's disguise))
|
("entity";TAPHIANS;PERSON;A group of people led by Mentes (Minerva's disguise))
|
("entity";GODDESS;GOD;The divine being (Minerva) who visited Telemachus disguised as Mentes, which Telemachus recognized in his heart)
|
("entity";SUITORS;PERSON;A group who returned to singing and dancing until evening before going to their own homes)
|
("entity";EURYCLEA;PERSON;A good old woman who accompanied Telemachus with torches, daughter of Ops, bought by Laertes when young)
|
("entity";OPS;PERSON;The father of Euryclea and son of Pisenor)
|
("enti


Book 0:  95%|█████████▌| 21/22 [03:32<00:11, 11.76s/it]

ults;6)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;A young man who is guided to his room by Euryclea, sits on his bed, removes his shirt, and thinks about his planned voyage and Minerva's advice while covered with a woollen fleece)
|
("entity";EURYCLEA;PERSON;An older woman who lights Telemachus to his room, loves him more than other women in the house, nursed him as a baby, folds his shirt, and secures his door)
|
("entity";MINERVA;GOD;A deity who gave counsel to Telemachus about a voyage, which he thinks about as he lies in bed)
|
("entity";LAERTES;PERSON;A man who bought Euryclea when she was young, respected her like his wife but did not take her to his bed out of fear of his wife's resentment)
|
("entity";WIFE;PERSON;Laertes' spouse whose resentment he feared, preventing him from taking Euryclea to his bed)
|
("relationship";TELEMACHUS;EURYCLEA;Euryclea was Telemachus' nurse when he was a baby, loves him more than other wome


Processing Books: 100%|██████████| 1/1 [03:43<00:00, 223.62s/it]


In [10]:
nodes[0]

{'record_type': 'entity',
 'entity_name': 'TELEMACHUS',
 'entity_type': 'PERSON',
 'entity_description': "A young man who is guided to his room by Euryclea, sits on his bed, removes his shirt, and thinks about his planned voyage and Minerva's advice while covered with a woollen fleece"}

In [11]:
relationships[0]

{'record_type': 'relationship',
 'source_entity': 'TELEMACHUS',
 'target_entity': 'EURYCLEA',
 'relationship_description': "Euryclea was Telemachus' nurse when he was a baby, loves him more than other women in the house, and now serves him by lighting his way and preparing his room",
 'relationship_strength': 9}

In [12]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (:`__Entity__`)
    RETURN 'entity' AS type, count(*) AS count
    UNION
    MATCH ()-[:RELATIONSHIP]->()
    RETURN 'relationship' AS type, count(*) AS count
    """
)
print([el.data() for el in data])

[{'type': 'entity', 'count': 83}, {'type': 'relationship', 'count': 188}]


In [13]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
    WHERE n.name = "ORESTES"
    RETURN n.description AS description"""
)
print([el.data()['description'] for el in data])

[["Agamemnon's son who killed Aegisthus", 'The son of Agamemnon who took revenge on Aegisthus', "A person praised for killing his father's murderer Aegisthus, cited as an example for Telemachus"]]


In [31]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[:RELATIONSHIP]-(m:__Entity__)
    WITH n,m, count(*) AS countOfRels
    WHERE n.name = "NEPTUNE"
    MATCH (n)-[r:RELATIONSHIP]-(m)
    RETURN n.name AS source, m.name AS target, countOfRels, collect(r.description) AS descriptions
    """
    )

for i in [el.data() for el in data]:
    print(i)

{'source': 'NEPTUNE', 'target': 'POLYPHEMUS', 'countOfRels': 1, 'descriptions': ["Neptune is Polyphemus' father"]}
{'source': 'NEPTUNE', 'target': 'ULYSSES', 'countOfRels': 2, 'descriptions': ['Neptune is furious with Ulysses for blinding Polyphemus and prevents him from returning home', 'Neptune persecuted Ulysses and prevented him from returning home']}
{'source': 'NEPTUNE', 'target': 'ETHIOPIANS', 'countOfRels': 1, 'descriptions': ['Neptune visited the Ethiopians to accept a hecatomb of sheep and oxen']}


In [21]:
candidates_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (e:__Entity__) WHERE size(e.description) > 1 and e.name = "AEGISTHUS"
    RETURN e.name AS entity_name, e.description AS description_list"""
)

In [32]:
candidates_to_summarize

[<Record entity_name='AEGISTHUS' description_list=["A man who had an affair with Agamemnon's wife and killed Agamemnon, later killed by Orestes", "A man who ignored Mercury's warnings, killed Agamemnon, and was subsequently killed by Orestes", "The murderer of Orestes' father who was killed by Orestes"]>]